In [76]:
import numpy as np
import pandas as pd
np.set_printoptions(precision=6, suppress=True)

# Решение задачи Дирихле:
## Известно, что $u(x,y) = 2x^3y^3$
## Оно является решением задачи Дирихле:
### $\frac{\partial^2 u}{\partial x^2} + \frac{\partial^2 u}{\partial y^2} = -(\underbrace{-12xy(x^2+y^2)}_{f(x,y)})$
### $u(0,y) \equiv 0$
### $u(1,y) = 2y^3$
### $u(x,0) \equiv 0$
### $u(x,1) = 2x^3$

In [77]:
N = 11
a_x = 0
b_x = 1
a_y = 0
b_y = 1
h = (b_x-a_x)/(N - 1) # ширина сетки вместе с граничными узлами
iN = N - 2 # ширина сетки без граничных узлов

f = lambda x,y: -12*x*y*(x*x+y*y)
u1y = lambda y: 2*y**3
ux1 = lambda x: 2*x**3
u_expected = lambda x,y: 2*x**3*y**3

### Задаём начальное приближение с помощью начальных данных

In [78]:
def init_U(N):
    U = np.zeros(shape = (N, N))
    for i in range(N):
        U[i, N - 1] = ux1(i*h)
    for j in range(N):
        U[N - 1, j] = u1y(j*h)
    return U

def build_init_expected_and_F(N, h, u_expected, ux1, u1y, f):
    U = init_U(N)

    U_expected = U.copy() # задаём границы
    U_expected[1:N-1,1:N-1] = np.array([[u_expected(i*h,j*h) for j in range(1, N-1)] for i in range(1, N-1)])
    F = np.array([[f(i*h, j*h) for j in range(1, N-1)] for i in range(1,N-1)])
    return {'init': U, 'expected': U_expected, 'F': F}

In [79]:
class DESolver:
    def __init__(self, grid_step, N, eps = 1e-5):
        self.h = grid_step
        self.methods = {'iteration': self.iteration_method_iter,
                        'seidel': self.seidel_method_iter,
                        'over-relaxation': self.overrelaxation_method_iter}
        self.methods_params = {'iteration'      : {'min_iters': 2*np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)},
                               'seidel'         : {'min_iters': np.log(1/eps)/(np.pi*h)**2,
                                                'spectral_rad': np.cos(np.pi * h)**2},
                               'over-relaxation': {'min_iters': 2*np.log(1/eps)/(np.pi*h),
                                                'spectral_rad': np.cos(np.pi * h),
                                                'opt_parameter': 2/(1 + np.sin(np.pi * h))}}
        self.N = N
        self.iN = N - 2
        C = np.matrix([[4, -1] + (N-2)*[0]] +
              [ k*[0] + [-1, 4, -1] + (N - 3 - k)*[0] for k in range(0, N - 2)] +
              [(N-2)*[0] + [-1, 4]], dtype=float)
        self.mmatrix = np.block([[C, -np.eye(N = N)] + (N-2)*[np.zeros( shape = (N, N) )]] +
              [ k*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C, -np.eye(N = N)] + (N - 3 - k)*[np.zeros( shape = (N, N) )] for k in range(0, N - 2)] +
              [(N-2)*[np.zeros( shape = (N, N) )] + [-np.eye(N = N), C]]) / grid_step**2
        self.lastsol = np.zeros(shape = (N, N))
        self.iter_diffs = np.zeros(shape = 3) # Хранит разницу приближений последних трёх итераций

    def __method_iter(self, U_curr, F, method_name, method_params):
        return self.methods[method_name](U_curr, F, params = method_params)

    def iteration_method_iter(self, U, F, params = None):
        for i in range(1, self.iN+1):
            for j in range(1, self.iN+1):
               self.lastsol[i, j] = 0.25*(U[i-1, j]+U[i+1,j]+U[i, j-1]+U[i, j+1]+F[i-1,j-1]*self.h**2)
        return self.lastsol

    def seidel_method_iter(self, U, F,params = None):
        for k in range(1, self.iN+1):
            for i in range(k, self.iN+1):
                self.lastsol[i, k] = 0.25*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
            for j in range(k+1, self.iN+1):
                self.lastsol[k, j] = 0.25*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
        return self.lastsol

    def overrelaxation_method_iter(self, U, F,params):
            for k in range(1, self.iN+1):
                for i in range(k, self.iN+1):
                    self.lastsol[i, k] = (1-params['opt'])*U[i,k] + 0.25*params['opt']*(self.lastsol[i-1, k]+U[i+1,k]+self.lastsol[i, k-1]+U[i, k+1]+F[i-1,k-1]*self.h**2)
                for j in range(k+1, self.iN+1):
                    self.lastsol[k, j] = (1-params['opt'])*U[k,j] + 0.25*params['opt']*(self.lastsol[k-1, j]+U[k+1,j]+self.lastsol[k, j-1]+U[k, j+1]+F[k-1,j-1]*self.h**2)
            return self.lastsol

    def __calculate_props(self, U, U_expected, F, iter, init_error, init_discrepancy, apost_est_coef):
        iter_diff = np.max(np.abs(U - self.lastsol))
        self.iter_diffs[iter % 3] = iter_diff
        apost_est = apost_est_coef * iter_diff
        spec_rad_est = ' ' if iter < 2 else np.sqrt(self.iter_diffs[iter % 3]/ self.iter_diffs[(iter - 2) % 3] )

        U_reshaped = np.reshape(self.lastsol, newshape=(self.N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix @ U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(self.N, self.N))[1:self.iN+1, 1:self.iN+1]

        error = np.max(np.abs(self.lastsol - U_expected))
        discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        rel_discrepancy = discrepancy / init_discrepancy
        rel_error = error / init_error

        return [iter + 1, discrepancy, rel_discrepancy,
                             error, rel_error, iter_diff, apost_est, spec_rad_est]

    def solve(self, U_init, U_expected, F, eps = 1e-5, method = 'iteration', method_parameters = None):
        if method not in ['iteration', 'seidel', 'over-relaxation']:
            return None
        else:
            self.iter_diffs = np.zeros(shape = 3)
            if method_parameters is None and method == 'over-relaxation':
                method_parameters = {'opt': self.methods_params['over-relaxation']['opt_parameter']}

        apost_est_coef = self.methods_params[method]['spectral_rad']/ (1 - self.methods_params[method]['spectral_rad'])
        init_error = np.max(np.abs(U_init - U_expected))
        U = U_init.copy()
        self.lastsol = U.copy()

        U_reshaped = np.reshape(U, newshape=(self.N**2, 1)) # преобразуем матрицу сетки к вектору
        AU = self.mmatrix@U_reshaped
        AU_reshaped_cut = np.reshape(AU, newshape=(self.N, self.N))[1:self.iN+1, 1:self.iN+1]

        init_discrepancy = np.max(np.abs(AU_reshaped_cut - F))
        output_list = list()
        iter = 0
        rel_discrepancy = 1

        while rel_discrepancy > eps and iter < self.methods_params[method]['min_iters']:
            U_new = self.__method_iter(U, F, method, method_parameters)

            curr_iter_props = self.__calculate_props(U, U_expected, F, iter, init_error, init_discrepancy, apost_est_coef)
            output_list+=[ curr_iter_props ]

            U = U_new.copy()
            rel_discrepancy = curr_iter_props[2]
            iter+=1
                                                                                      
        output_list += [[' ', 'min_iters:', self.methods_params[method]['min_iters'], ' ', ' method:', method, ' ', ' ']]
        return {'solution': U ,'output': pd.DataFrame(data = output_list,
                                                      columns = ['k', 'Curr. discr.', 'Rel. discr.', 'Curr. error', 'Rel. error', '$ \lVert U_k - U_{k-1} \rVert $', 'Apost. est.', '$\hat{\rho}_k$']).set_index(keys = 'k')}


In [80]:
solver = DESolver(h, N)
initials = build_init_expected_and_F(N, h, u_expected,ux1, u1y, f)
U_init = initials['init']
U_expected = initials['expected']
F = initials['F']

### Решение методом простой итерации

In [81]:
iter_summary = solver.solve(U_init, method = 'iteration', U_expected = U_expected, F = F)
solU_i = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,81.1988,0.294355,0.548864,0.516392,0.689634,13.400781,
2,45.2768,0.164133,0.446016,0.419629,0.202997,3.944583,
3,28.25815,0.102439,0.332824,0.313134,0.113192,2.199516,0.405134
4,24.2665,0.087969,0.274983,0.258715,0.070645,1.372762,0.589925
5,14.478205,0.052485,0.219805,0.206801,0.060666,1.17885,0.732092
...,...,...,...,...,...,...,...
134,0.003265,0.000012,0.000144,0.000136,0.000008,0.000159,0.951052
135,0.002953,0.000011,0.000136,0.000128,0.000008,0.000159,0.951057
136,0.002953,0.000011,0.00013,0.000123,0.000007,0.000143,0.951053


### Решение методом Зейделя

In [82]:
iter_summary = solver.solve(U_init, method = 'seidel', U_expected = U_expected, F = F)
solU_s = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,81.741131,0.296321,0.564035,0.530666,0.817411,7.742631,
2,52.139987,0.189013,0.422256,0.397275,0.2607,2.469385,
3,25.085994,0.090940,0.289813,0.272667,0.162229,1.536658,0.445496
4,15.59163,0.056521,0.238097,0.224011,0.08663,0.820574,0.576454
5,10.922248,0.039594,0.191427,0.180102,0.054611,0.517285,0.580198
...,...,...,...,...,...,...,...
71,0.00344,0.000012,0.000163,0.000153,0.000017,0.000163,0.904508
72,0.003111,0.000011,0.000147,0.000139,0.000016,0.000147,0.904508
73,0.002814,0.000010,0.000133,0.000125,0.000014,0.000133,0.904508


### Решение методом верхней релаксации

In [83]:
ovopt = 2/(1 + np.sin(np.pi * h))
iter_summary = solver.solve(U_init, method = 'over-relaxation', U_expected = U_expected, F = F)
solU_or = iter_summary['solution']
iter_summary['output'][::]

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,188.549832,0.683514,0.612457,0.576223,1.364359,26.511862,
2,102.888676,0.372983,0.340329,0.320195,0.499472,9.705599,
3,79.553441,0.288390,0.237854,0.223782,0.305825,5.942714,0.473448
4,50.003605,0.181269,0.175503,0.16512,0.191153,3.714424,0.618635
5,37.635711,0.136434,0.129915,0.122229,0.137264,2.667277,0.669949
6,24.625723,0.089271,0.10025,0.094319,0.095508,1.855894,0.706856
7,18.357216,0.066547,0.079239,0.074551,0.072131,1.401638,0.72491
8,12.182867,0.044164,0.060826,0.057227,0.052331,1.016884,0.740217
9,8.997514,0.032617,0.048227,0.045374,0.040414,0.785314,0.748521


### Убедимся, что параметр по умолчанию является оптимальным

In [84]:
solver.solve(U_init, method = 'seidel', U_expected = U_expected, F = F, method_parameters={'opt': ovopt-0.1})['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,81.741131,0.296321,0.564035,0.530666,0.817411,7.742631,
2,52.139987,0.189013,0.422256,0.397275,0.2607,2.469385,
3,25.085994,0.090940,0.289813,0.272667,0.162229,1.536658,0.445496
4,15.59163,0.056521,0.238097,0.224011,0.08663,0.820574,0.576454
5,10.922248,0.039594,0.191427,0.180102,0.054611,0.517285,0.580198
...,...,...,...,...,...,...,...
71,0.00344,0.000012,0.000163,0.000153,0.000017,0.000163,0.904508
72,0.003111,0.000011,0.000147,0.000139,0.000016,0.000147,0.904508
73,0.002814,0.000010,0.000133,0.000125,0.000014,0.000133,0.904508


In [85]:
solver.solve(U_init, method = 'seidel', U_expected = U_expected, F = F, method_parameters={'opt': ovopt+0.1})['output']

,Curr. discr.,Rel. discr.,Curr. error,Rel. error,$ \lVert U_k - U_{k-1} \rVert $,Apost. est.,$\hat{\rho}_k$
k,,,,,,,
1,81.741131,0.296321,0.564035,0.530666,0.817411,7.742631,
2,52.139987,0.189013,0.422256,0.397275,0.2607,2.469385,
3,25.085994,0.090940,0.289813,0.272667,0.162229,1.536658,0.445496
4,15.59163,0.056521,0.238097,0.224011,0.08663,0.820574,0.576454
5,10.922248,0.039594,0.191427,0.180102,0.054611,0.517285,0.580198
...,...,...,...,...,...,...,...
71,0.00344,0.000012,0.000163,0.000153,0.000017,0.000163,0.904508
72,0.003111,0.000011,0.000147,0.000139,0.000016,0.000147,0.904508
73,0.002814,0.000010,0.000133,0.000125,0.000014,0.000133,0.904508


### видим, что алгоритму приходится выполнять больше итераций, чтобы достичь нужной точности


## Сравним полученные разными методами решения с точным на крупной сетке

In [86]:
largegrid_solver = DESolver(0.2, 6)
largegrid_initials = build_init_expected_and_F(6, 0.2, u_expected, ux1, u1y, f)
lgU_init = largegrid_initials['init']
lgU_expected = largegrid_initials['expected']
lgF = largegrid_initials['F']

lgsolU_i = largegrid_solver.solve(lgU_init, lgU_expected, lgF, method = 'iteration')['solution']
lgsolU_s = largegrid_solver.solve(lgU_init, lgU_expected, lgF, method = 'seidel')['solution']
lgsolU_or = largegrid_solver.solve(lgU_init, lgU_expected, lgF, method = 'over-relaxation')['solution']

In [87]:
print(lgU_expected)

print(lgsolU_i)

print(lgsolU_s)

print(lgsolU_or)

[[0.       0.       0.       0.       0.       0.      ]
 [0.       0.000128 0.001024 0.003456 0.008192 0.002   ]
 [0.       0.001024 0.008192 0.027648 0.065536 0.016   ]
 [0.       0.003456 0.027648 0.093312 0.221184 0.054   ]
 [0.       0.008192 0.065536 0.221184 0.524288 0.128   ]
 [0.       0.002    0.016    0.054    0.128    0.25    ]]
[[ 0.        0.        0.        0.        0.        0.      ]
 [ 0.       -0.022195 -0.043622 -0.058805 -0.052847  0.002   ]
 [ 0.       -0.043622 -0.08581  -0.11571  -0.102361  0.016   ]
 [ 0.       -0.058805 -0.11571  -0.155962 -0.134007  0.054   ]
 [ 0.       -0.052847 -0.102361 -0.134007 -0.101308  0.128   ]
 [ 0.        0.002     0.016     0.054     0.128     0.25    ]]
[[ 0.        0.        0.        0.        0.        0.      ]
 [ 0.       -0.022195 -0.043622 -0.058805 -0.052848  0.002   ]
 [ 0.       -0.043622 -0.08581  -0.115711 -0.102362  0.016   ]
 [ 0.       -0.058805 -0.115711 -0.155963 -0.134008  0.054   ]
 [ 0.       -0.052848 -0.1